# 

# Automating processes:
----
(interactive version -> *.ipynb*)

## Importing libraries:

In [1]:
import numpy as np
import pandas as pd
import selenium as sn
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get("https://www.google.com")